In [1]:
from python_test import *
import pandas as pd
from scipy import ndimage

c:\Users\workstation3\anaconda3\envs\cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define variables

In [277]:
source_path = "//storage3.ad.scilifelab.se/alm/BrismarGroup/Hanna/Master2023/Recordings/2023-09-01/"
masks_name = "hep-oua-ftrc2-dl20-btl15-diam518_masks.tif"
images_name = "hep-oua.tif"
model_path = "C:/Users/workstation3/Documents/Hannas_models/for-the-report-cyto2"
save_path = "//storage3.ad.scilifelab.se/alm/BrismarGroup/Hanna/Master2023/Recordings/Intensity-data/hep-oua/"
save_name = "0901.csv"
occurrence_limit = 50   # Percentage of images a cell has to be in order to be accounted for
T = 10 # Period between images (seconds)
max_dt = 30     # Maximum time shift allowed for the correlation calculations (time steps)

# If generating new masks
# diam = 21     # Get this from the GUI
# dist_limit = 9    # Slightly less than half average diameter
# backtrack_limit = 15

Load masks and images

In [278]:
images = open_image_stack(source_path + images_name)

100%|██████████| 360/360 [00:00<00:00, 631.23it/s]


In [279]:
# Generate masks
# masks = get_segmentation(images, model_path, diam=diam)
# tracked_masks = get_tracked_masks(masks, dist_limit=dist_limit, backtrack_limit=backtrack_limit, save=False)
# imwrite(source_path+masks_name,tracked_masks)   # Save the masks

# Open pregenerated masks
tracked_masks = imread(source_path + masks_name)

100%|██████████| 360/360 [00:04<00:00, 79.53it/s] 


Get intensities and put in a Pandas DataFrame

In [280]:
common_cells, counts = get_common_cells(tracked_masks, occurrence=occurrence_limit) # Get the cells to include in the intensity measurements
df = pd.DataFrame(columns=common_cells)     # Generate DataFrame for storing the information

In [281]:
intensities = {}
for c in common_cells:
    intensities[c] = get_cell_intensities(c, tracked_masks, images, T=T)

start<0:  [0 1 2 3 4 5]
start<0:  [0 1 2 3 4 5 6 7 8]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]


c:\Users\workstation3\anaconda3\envs\cellpose\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\workstation3\anaconda3\envs\cellpose\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
start<0:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28

In [282]:
row = pd.Series(intensities,name="Intensities")
df = df.append(row)

C:\Users\workstation3\AppData\Local\Temp\ipykernel_12432\3077831015.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)


Get centers of mass and append to DataFrame

In [283]:
COMs = {}
for c in common_cells:
    for im in range(len(tracked_masks)):
        if np.any(tracked_masks[im]==c):
            COMs[c] = list(get_centers_of_mass(tracked_masks[im], c)[0])
            break

In [284]:
row = pd.Series(COMs,name="Center of mass")
df = df.append(row)

C:\Users\workstation3\AppData\Local\Temp\ipykernel_12432\3237797314.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)


Get nearest neighbors/border values and append to DataFrame

In [285]:
def filtered_masks(masks, common_cells):
    """Set all labels in masks not in common_cells to 0."""
    filter = np.isin(masks, common_cells)
    masks[~filter] = 0
    return masks

In [286]:
def extract_border_values(flt_masks, cell_lbl):
    def get_border_values(mask, cell_lbl):
        padded_mask = np.pad(mask, 1, "constant", constant_values=0) # Wherever there are no neighboring cells, we 
                                                                     # want zeroes, also on edges of image
        fltr = ndimage.binary_dilation(padded_mask==cell_lbl, structure=np.ones((3, 3)))
        bv = padded_mask[fltr]
        bv = bv[bv!=cell_lbl]
        return bv

    bv_count = 10
    bv_final = []
    for m in flt_masks:
        if not np.any(m==cell_lbl):
            continue
        bv = get_border_values(m,cell_lbl)
        if not np.any(bv==0):
            return bv
        q = len(bv[bv==0])/len(bv) # Check how much of the border is 0
        if q<0.05: # We're satisfied if 95% of the border is nonzero
            return bv
        if q<bv_count:
            bv_count = q
            bv_final = bv
    return bv_final

In [287]:
fltr_masks = filtered_masks(tracked_masks, common_cells)
border_values = {}
for c in common_cells:
    border_values[c] = list(extract_border_values(fltr_masks, c))

In [288]:
row = pd.Series(border_values,name="Border values")
df = df.append(row)

C:\Users\workstation3\AppData\Local\Temp\ipykernel_12432\3697260902.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)


Calculate normalized cross correlation with bordering cells, weighted with the amount of border shared

In [289]:
def get_ncc_for_bv(df, cell_lbl, max_dt):
    bvs = np.array(df.loc["Border values", cell_lbl])
    bvs = bvs[bvs!=0]
    bvs_uniq = np.unique(bvs)
    ncc = 0
    dts = 0
    for bv in bvs_uniq:
        print(bv)
        xcorr, dt = get_cc(np.array(df.loc["Intensities", cell_lbl]),
                           np.array(df.loc["Intensities", bv]), max_dt)
        w = len(bvs[bvs==bv])/len(bvs[bvs!=0])
        ncc += np.max(xcorr)*w
        dts += dt[np.argmax(xcorr)]*w
    return ncc, dts

In [290]:
cross_correlations = {}
time_diffs = {}
for c in common_cells:
    cross_correlations[c], time_diffs[c] = get_ncc_for_bv(df, c, max_dt)

9
13
116
132
6
8
15
18
119
139
5
15
120
5
11
18
119
139
10
13
16
19
116
9
12
17
116
8
12
17
18
24
26
8
10
11
17
24
1
9
14
16
23
128
132
1
13
23
128
132
5
6
119
120
122
9
13
19
23
25
10
11
12
19
24
28
8
11
20
26
9
10
16
17
24
25
28
31
39
119
122
123
13
16
25
36
128
129
11
17
19
26
28
30
37
16
19
23
28
36
11
18
20
24
30
31
19
24
25
30
37
44
15
35
120
122
20
24
26
28
37
42
20
26
30
39
42
29
39
51
57
122
123
147
23
25
28
44
45
129
28
30
44
53
124
23
45
128
129
143
20
31
35
42
55
123
141
147
30
31
39
55
124
28
36
37
45
53
125
36
38
44
56
109
125
129
143
38
109
35
57
58
147
37
44
62
65
124
125
39
42
62
71
72
124
141
45
66
69
109
125
51
58
67
75
141
147
51
57
67
48
56
69
74
109
37
53
55
65
71
76
124
53
62
66
76
77
125
56
65
69
77
82
125
57
58
75
81
56
60
66
74
82
84
85
89
109
55
62
72
76
80
55
71
78
80
141
60
69
89
57
67
78
81
86
141
62
65
71
77
80
83
87
65
66
76
82
83
91
93
72
75
80
86
88
71
72
76
78
87
88
90
67
75
86
66
77
85
91
94
76
77
87
93
69
85
89
95
113
69
82
84
94
95
75
78
81
88
96
9

In [291]:
row = pd.Series(cross_correlations,name="Weighted max NCC for border values")
df = df.append(row)
row = pd.Series(time_diffs,name="Weighted time difference at max NCC for border values")
df = df.append(row)

C:\Users\workstation3\AppData\Local\Temp\ipykernel_12432\27558605.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)
C:\Users\workstation3\AppData\Local\Temp\ipykernel_12432\27558605.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)


Find maximum frequency

In [292]:
def get_max_freq(df, T):
    fouriermax = {}
    for c in df.columns:
        fft = np.fft.fft(df.loc["Intensities", c])
        freqs = np.fft.fftfreq(len(df.loc["Intensities", c]), T)
        fft_filtered = fft*(freqs>(3/(3600))) #Remove all periods longer than 20 min  
        fouriermax[c] = np.abs(freqs[np.argmax(fft_filtered)])
    return fouriermax

In [293]:
fouriermax = get_max_freq(df, T)

In [294]:
row = pd.Series(fouriermax,name="Most prominent frequency")
df = df.append(row)

C:\Users\workstation3\AppData\Local\Temp\ipykernel_12432\2216402935.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row)


Save the data

In [295]:
df.to_csv(save_path+save_name)

In [219]:
df

,1,2,3,4,5,6,7,8,9,10,...,253,257,277,279,280,282,283,290,296,299
Intensities,"[3039.4683908045977, 3123.3896551724138, 3182....","[4942.464749536178, 5094.0151209677415, 5223.6...","[2719.905923344948, 2791.4030710172747, 2776.7...","[3106.7119205298013, 3167.6571274298058, 3151....","[2951.5458052073286, 3036.950835791544, 3030.4...","[3132.6515880654474, 3266.8136042402825, 3264....","[3397.6852459016395, 3415.0420168067226, 3421....","[4146.780353200883, 4221.463626492942, 4206.36...","[3876.327981651376, 4003.7420382165606, 3999.0...","[3899.67081604426, 3957.9807407407407, 3963.89...",...,"[5631.604417670683, 5842.9859437751, 5896.5883...","[3090.875816993464, 3184.264705882353, 3210.95...","[2943.1539235412474, 2985.5266599597585, 2981....","[2547.3813732154995, 2604.7634262406527, 2610....","[3054.296108291032, 3118.7901861252117, 3101.7...","[5347.320137693631, 5471.621342512909, 5464.43...","[3743.3877094972067, 3780.630167597765, 3792.0...","[4516.92, 4737.61, 4727.56, 4770.476, 4803.104...","[3669.12852969815, 3731.3495618305747, 3710.56...","[3432.812834224599, 3495.301247771836, 3496.12..."
Center of mass,"[10.96264367816092, 50.28927203065134]","[19.4860853432282, 90.77365491651206]","[9.52787456445993, 168.301393728223]","[25.166114790286976, 197.18818984547462]","[13.216972034715525, 234.46287367405978]","[13.558229066410009, 270.18479307025984]","[23.41639344262295, 291.7049180327869]","[25.107064017660043, 312.14348785871965]","[12.763761467889909, 332.7178899082569]","[11.730290456431534, 363.21853388658366]",...,"[460.2409638554217, 351.2429718875502]","[59.290849673202615, 59.50326797385621]","[426.9517102615694, 16.419014084507044]","[49.65669612508498, 30.6512576478586]","[160.93231810490693, 62.179357021996616]","[313.66179001721173, 136.97160068846816]","[371.6368715083799, 46.33296089385475]","[503.962, 340.702]","[165.8860759493671, 82.77312560856865]","[400.05882352941177, 499.42245989304814]"
Border values,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6, 6, 6, 6, 0, 6, 0, 6, 6, 8, 6, 6, 8, 8, 6, ...","[0, 0, 0, 0, 0, 0, 6, 9, 6, 9, 6, 9, 6, 9, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[215, 214, 214, 214, 214, 214, 214, 214, 214, ...","[279, 0, 17, 17, 17, 17, 17, 17, 17, 17, 279, ...","[283, 283, 283, 283, 283, 283, 283, 283, 283, ...","[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45,...","[140, 122, 122, 122, 122, 122, 122, 140, 140, ...","[154, 154, 154, 154, 154, 154, 164, 164, 154, ...","[240, 228, 228, 240, 228, 228, 240, 240, 228, ...","[58, 58, 66, 58, 58, 58, 58, 66, 66, 58, 58, 5...","[185, 185, 185, 185, 185, 185, 185, 185, 185, ..."
Weighted max NCC for border values,0.861952,0.87857,0.785191,0.688938,0.848491,0.922129,0.916112,0.881596,0.775534,0.488759,...,0.773455,0.692021,0.399,0.823246,0.671075,0.786156,0.479752,0.878053,0.82859,0.771112
Weighted time difference at max NCC for border values,-9.354839,24.836957,3.052632,-4.72973,0.0,0.247706,0.0,-2.17284,0.325,0.0,...,3.323529,0.0,6.706897,3.716867,14.788571,-12.565445,0.5,0.0,18.506757,9.3625
Most prominent frequency,0.003073,0.003073,0.001117,0.002793,0.002514,0.002514,0.009218,0.001955,0.001676,0.001676,...,0.002514,0.002793,0.003073,0.003073,0.002235,0.000838,0.003631,0.001117,0.003911,0.000838
